In [1]:
import requests
import json
import math
import ccd
import base64
import xarray as xr
import pandas as pd
import numpy as np
from datetime import datetime
from test.shared import snap, rainbow
import os

In [2]:
# detect results for x, y: -2094435, 1681005 took 2.455234 seconds to generate  VERIFIED 2.6 seconds locally
# detect results for x, y: -2033595, 1685955 took 1.879693 seconds to generate
# detect results for x, y: -2098875, 1694895 took 0.986887 seconds to generate  VERIFIED 1.8 seconds locally
pixl_x     = -2094435
pixl_y     = 1681005
pixl_x = -2098875
pixl_y = 1694895
chips_host = os.getenv('LCMAP_CHIP_HOST')

inputs_url = "{chips_host}?x={x}&y={y}\
&acquired=1982-01-01/2015-12-31\
&ubid=LANDSAT_4/TM/SRB1\
&ubid=LANDSAT_4/TM/SRB2\
&ubid=LANDSAT_4/TM/SRB3\
&ubid=LANDSAT_4/TM/SRB4\
&ubid=LANDSAT_4/TM/SRB5\
&ubid=LANDSAT_4/TM/BTB6\
&ubid=LANDSAT_4/TM/SRB7\
&ubid=LANDSAT_4/TM/PIXELQA\
&ubid=LANDSAT_5/TM/SRB1\
&ubid=LANDSAT_5/TM/SRB2\
&ubid=LANDSAT_5/TM/SRB3\
&ubid=LANDSAT_5/TM/SRB4\
&ubid=LANDSAT_5/TM/SRB5\
&ubid=LANDSAT_5/TM/BTB6\
&ubid=LANDSAT_5/TM/SRB7\
&ubid=LANDSAT_5/TM/PIXELQA\
&ubid=LANDSAT_7/ETM/SRB1\
&ubid=LANDSAT_7/ETM/SRB2\
&ubid=LANDSAT_7/ETM/SRB3\
&ubid=LANDSAT_7/ETM/SRB4\
&ubid=LANDSAT_7/ETM/SRB5\
&ubid=LANDSAT_7/ETM/BTB6\
&ubid=LANDSAT_7/ETM/SRB7\
&ubid=LANDSAT_7/ETM/PIXELQA\
&ubid=LANDSAT_8/OLI_TIRS/SRB2\
&ubid=LANDSAT_8/OLI_TIRS/SRB3\
&ubid=LANDSAT_8/OLI_TIRS/SRB4\
&ubid=LANDSAT_8/OLI_TIRS/SRB5\
&ubid=LANDSAT_8/OLI_TIRS/SRB6\
&ubid=LANDSAT_8/OLI_TIRS/SRB7\
&ubid=LANDSAT_8/OLI_TIRS/BTB10\
&ubid=LANDSAT_8/OLI_TIRS/PIXELQA".format(chips_host=chips_host, x=pixl_x, y=pixl_y)

dates           = [i.split('=')[1] for i in inputs_url.split('&') if 'acquired=' in i][0]
chips_url       = inputs_url.split('?')[0]
specs_url       = chips_url.replace('/chips', '/chip-specs')
querystr_list   = inputs_url.split('?')[1].split('&')
requested_ubids = [i.replace('ubid=', '') for i in querystr_list if 'ubid=' in i]

In [3]:
# get chip id
chip_x, chip_y = snap(pixl_x, pixl_y)
xindex = int((pixl_x - chip_x) / 30)
yindex = int((chip_y - pixl_y) / 30)

In [4]:
def return_xarray(filepath):
    with open(filepath, 'r') as f:
        _rjson = json.loads(f.read())
    return xr.Dataset.from_dict(_rjson)

def save_xarray(xd, filepath):
    with open(filepath, 'w') as f:
        f.write(json.dumps(xd.to_dict()))        
        
def fetch_rainbow_data(chip_x, chip_y, save_path="test/resources"):        
    # does chip json exist? if not, attempt retrieval
    json_filename = "{}/saved_rainbow-{}-{}.json".format(save_path, chip_x, chip_y)
    if os.access(json_filename, os.F_OK):
        rbow = return_xarray(json_filename)
    else:
        rbow = rainbow(chip_x, chip_y, dates, specs_url, chips_url, requested_ubids)
        save_xarray(rbow, json_filename)
    return rbow    

In [5]:
rbow2 = fetch_rainbow_data(chip_x, chip_y)

In [6]:
def dtstr_to_ordinal(dtstr, iso=True):
    """ Return ordinal from string formatted date"""
    _fmt = '%Y-%m-%dT%H:%M:%SZ' if iso else '%Y-%m-%d %H:%M:%S'
    _dt = datetime.strptime(dtstr, _fmt)
    return _dt.toordinal()

def detect(rainbow, x, y):
    """ Return results of ccd.detect for a given stack of data at a particular x and y """
    try:
        # Beware: rainbow contains stacks of row-major two-dimensional arrays
        # for each band of data. These variables are used to make the order
        # of access clear.
        _start = datetime.now()
        row, col = y, x
        rainbow_date_array = np.array(rainbow['t'].values)
        
        result =  ccd.detect([dtstr_to_ordinal(str(pd.to_datetime(i)), False) for i in rainbow_date_array],
                             np.array(rainbow['blue'].values[:, row, col]),
                             np.array(rainbow['green'].values[:, row, col]),
                             np.array(rainbow['red'].values[:, row, col]),
                             np.array(rainbow['nir'].values[:, row, col]),
                             np.array(rainbow['swir1'].values[:, row, col]),
                             np.array(rainbow['swir2'].values[:, row, col]),
                             np.array(rainbow['thermal'].values[:, row, col]),
                             np.array(rainbow['cfmask'].values[:, row, col], dtype=int),
                             params={})
        print("detect took {} seconds to run".format((datetime.now()-_start).total_seconds()))
    except Exception as e:
        raise Exception(e)

In [7]:
rbow2_results = detect(rbow2, xindex, yindex)

detect took 1.94772 seconds to run
